In [12]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import numpy as np
from io import StringIO

In [13]:
class Database:
    def __init__(self, DB_USER, DB_PASS, DB_NAME, DB_HOST, DB_PORT) -> None:
        self.conn = psycopg2.connect(database=DB_NAME,
                            user=DB_USER,
                            password=DB_PASS,
                            host=DB_HOST,
                            port=DB_PORT)
        self.cur = self.conn.cursor()
        
    def execute_dql(self, query):
        try:
            self.cur.execute(query)
            rows = self.cur.fetchall()
            columns = [desc[0] for desc in self.cur.description]
            return rows, columns 
        except (Exception, psycopg2.DatabaseError) as error:
            print(f"An error occurred: {error}")
            self.conn.rollback()  
            return False
        
    def execute_dml(self, query):
        try:
            self.cur.execute(query)
            self.conn.commit()
            return True
        except (Exception, psycopg2.DatabaseError) as error:
            print(f"An error occurred: {error}")
            self.conn.rollback()  
            return False
        
    def copy_from_dataframe(self, df, table_name):
        try:
            buffer = StringIO()
            df.to_csv(buffer, sep=';', index=False, header=False)
            buffer.seek(0)
            self.cur.copy_expert(f"COPY {table_name} FROM STDIN WITH (FORMAT csv, DELIMITER ';');", buffer)
            self.conn.commit()
            print("Data copied successfully!")
        except (Exception, psycopg2.DatabaseError) as error:
            print(f"An error occurred during COPY: {error}")
            self.conn.rollback()
            return False
        
    def close(self):
        if self.cur:
            self.cur.close()
        if self.conn:
            self.conn.close()

In [14]:
from dotenv import load_dotenv
import os

load_dotenv()

db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

database = Database(db_user, db_password, db_name, db_host, db_port)

In [16]:
input_df = pd.read_excel("./FileMEV.xlsx")
input_df.dtypes

ME_CODE              object
ME_PERIOD    datetime64[ns]
ME_VAL              float64
dtype: object

In [15]:
#CREATE INPUT TABLE
create_input_table_query = """
    CREATE TABLE mev_table (
        ME_CODE VARCHAR(50),
        ME_PERIOD TIMESTAMP,
        ME_VAL DOUBLE PRECISION
    );
"""

database.execute_dml(create_input_table_query)


An error occurred: relation "mev_table" already exists



False

In [ ]:
#COPY XLSX INTO DB
database.copy_from_dataframe(input_df, "mev_table")

Data copied successfully!


In [19]:
#SELECT MEV_TABLE
rows, columns = database.execute_dql('SELECT * FROM mev_table;')

df = pd.DataFrame(rows, columns=columns)
df

,me_code,me_period,me_val
0,BI7DRR,2021-01-31,3.750000
1,DPOL_IMP_BCAD,2021-01-31,-6.579436
2,GDP_GROWTHYOY_BCAD,2021-01-31,0.000000
3,INF_YOY_BCAD,2021-01-31,1.552765
4,KURS_BCAD,2021-01-31,14030.000000
...,...,...,...
79,DPOL_IMP_BCAD,2022-02-28,25.431587
80,GDP_GROWTHYOY_BCAD,2022-02-28,0.000000
81,INF_YOY_BCAD,2022-02-28,2.055440
82,KURS_BCAD,2022-02-28,14367.000000


In [20]:
df['me_val'] = df['me_val'].apply(lambda x: np.log(x) if x > 0 else 0)
df

,me_code,me_period,me_val
0,BI7DRR,2021-01-31,1.321756
1,DPOL_IMP_BCAD,2021-01-31,0.000000
2,GDP_GROWTHYOY_BCAD,2021-01-31,0.000000
3,INF_YOY_BCAD,2021-01-31,0.440037
4,KURS_BCAD,2021-01-31,9.548953
...,...,...,...
79,DPOL_IMP_BCAD,2022-02-28,3.235992
80,GDP_GROWTHYOY_BCAD,2022-02-28,0.000000
81,INF_YOY_BCAD,2022-02-28,0.720490
82,KURS_BCAD,2022-02-28,9.572689


In [21]:
#CREATE OUTPUT TABLE

create_output_table_query = '''
CREATE TABLE IF NOT EXISTS log_mev_table (
    ME_CODE VARCHAR(50),
    ME_PERIOD TIMESTAMP,
    ME_VAL DOUBLE PRECISION
);
'''

database.execute_dml(create_output_table_query)

True

In [ ]:
#COPY XLSX INTO DB
database.copy_from_dataframe(df, "log_mev_table")

Data copied successfully!


In [23]:
#SELECT MEV_TABLE
rows, columns = database.execute_dql('SELECT * FROM log_mev_table;')

df = pd.DataFrame(rows, columns=columns)
df

,me_code,me_period,me_val
0,BI7DRR,2021-01-31,1.321756
1,DPOL_IMP_BCAD,2021-01-31,0.000000
2,GDP_GROWTHYOY_BCAD,2021-01-31,0.000000
3,INF_YOY_BCAD,2021-01-31,0.440037
4,KURS_BCAD,2021-01-31,9.548953
...,...,...,...
79,DPOL_IMP_BCAD,2022-02-28,3.235992
80,GDP_GROWTHYOY_BCAD,2022-02-28,0.000000
81,INF_YOY_BCAD,2022-02-28,0.720490
82,KURS_BCAD,2022-02-28,9.572689
